In [ ]:
import pandas as pd
import numpy as np

# Data Import and Initial Structure Overview

In [2]:
df = pd.read_csv(r'..\datasets\financial_fraud_detection_dataset.csv')

In [3]:
def optimize_memory(df):
    """
    Iterate through all columns and modify the data type
    to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage before optimization: {start_mem:.2f} MB')

    import pandas as pd
    
    for col in df.columns:
        col_type = df[col].dtype
    
        # Only attempt numeric downcasting if the column is numeric
        if pd.api.types.is_numeric_dtype(df[col]) and not pd.api.types.is_bool_dtype(df[col]):
            c_min = df[col].min()
            c_max = df[col].max()
            # Integer downcasting
            if pd.api.types.is_integer_dtype(df[col]):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            # Float downcasting
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        # Convert high-cardinality strings to category
        elif col_type is object:
            # Heuristic: convert if number of unique values is less than 50% of total
            if len(df[col].unique()) / len(df[col]) < 0.5:
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage after optimization: {end_mem:.2f} MB')
    print(f'Decreased by {100 * (start_mem - end_mem) / start_mem:.1f}%')
    return df

df = optimize_memory(df)

Memory usage before optimization: 653.27 MB
Memory usage after optimization: 543.59 MB
Decreased by 16.8%


In [4]:
df

,transaction_id,timestamp,sender_account,receiver_account,amount,transaction_type,merchant_category,location,device_used,is_fraud,fraud_type,time_since_last_transaction,spending_deviation_score,velocity_score,geo_anomaly_score,payment_channel,ip_address,device_hash
0,T100000,2023-08-22T09:22:43.516168,ACC877572,ACC388389,343.779999,withdrawal,utilities,Tokyo,mobile,False,NaN,NaN,-0.21,3,0.22,card,13.101.214.112,D8536477
1,T100001,2023-08-04T01:58:02.606711,ACC895667,ACC944962,419.649994,withdrawal,online,Toronto,atm,False,NaN,NaN,-0.14,7,0.96,ACH,172.52.47.194,D2622631
2,T100002,2023-05-12T11:39:33.742963,ACC733052,ACC377370,2773.860107,deposit,other,London,pos,False,NaN,NaN,-1.78,20,0.89,card,185.98.35.23,D4823498
3,T100003,2023-10-10T06:04:43.195112,ACC996865,ACC344098,1666.219971,deposit,online,Sydney,pos,False,NaN,NaN,-0.60,6,0.37,wire_transfer,107.136.36.87,D9961380
4,T100004,2023-09-24T08:09:02.700162,ACC584714,ACC497887,24.430000,transfer,utilities,Toronto,mobile,False,NaN,NaN,0.79,13,0.27,ACH,108.161.108.255,D7637601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,T5099995,2023-11-17T23:20:29.746144,ACC597319,ACC749300,10.870000,withdrawal,retail,Toronto,atm,False,NaN,1416.524292,-0.14,17,0.18,UPI,243.92.38.163,D4439579
4999996,T5099996,2023-09-23T11:23:20.659686,ACC749625,ACC709783,181.399994,payment,grocery,Sydney,atm,False,NaN,999.089722,-1.79,4,0.58,wire_transfer,28.252.18.249,D5029311
4999997,T5099997,2023-11-18T00:52:34.527092,ACC629492,ACC680736,12.540000,payment,utilities,New York,mobile,False,NaN,3871.583984,-0.30,6,0.99,card,111.199.174.121,D6333607
4999998,T5099998,2023-03-25T04:32:13.609837,ACC984720,ACC296935,376.290009,deposit,restaurant,Dubai,pos,False,NaN,-4096.765625,-1.43,5,0.32,wire_transfer,221.110.215.14,D1551203


In [5]:
# Initial data inspection
print("Data Shape:", df.shape)
print("\nData Info:")
df.info(verbose=True, memory_usage='deep', show_counts=True)

Data Shape: (5000000, 18)

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 18 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   transaction_id               5000000 non-null  object 
 1   timestamp                    5000000 non-null  object 
 2   sender_account               5000000 non-null  object 
 3   receiver_account             5000000 non-null  object 
 4   amount                       5000000 non-null  float32
 5   transaction_type             5000000 non-null  object 
 6   merchant_category            5000000 non-null  object 
 7   location                     5000000 non-null  object 
 8   device_used                  5000000 non-null  object 
 9   is_fraud                     5000000 non-null  bool   
 10  fraud_type                   179553 non-null   object 
 11  time_since_last_transaction  4103487 non-null  float32
 12  spen

In [6]:
# Check for class imbalance (crucial for fraud detection)
print("\nFraud Distribution:")
print(df['is_fraud'].value_counts(normalize=True))


Fraud Distribution:
is_fraud
False    0.964089
True     0.035911
Name: proportion, dtype: float64


# Feature Engineering

In [ ]:
# Temporal features
df['timestamp'] = pd.to_datetime(df['timestamp'], format='mixed')
df = df.sort_values('timestamp').reset_index(drop=True)

# Create integer codes for categorical columns used in rolling nunique
df['sender_account_codes'], _ = pd.factorize(df['sender_account'])
df['receiver_account_codes'], _ = pd.factorize(df['receiver_account'])
df['device_hash_codes'], _ = pd.factorize(df['device_hash'])

df['transaction_hour'] = df['timestamp'].dt.hour
df['transaction_day_of_week'] = df['timestamp'].dt.dayofweek
df['transaction_month'] = df['timestamp'].dt.month
df['is_weekend'] = (df['transaction_day_of_week'] >= 5).astype(int)
df['hour_sin'] = np.sin(2 * np.pi * df['transaction_hour']/23.0)
df['hour_cos'] = np.cos(2 * np.pi * df['transaction_hour']/23.0)

# Moved features before df_indexed is defined
df['ip_subnet'] = df['ip_address'].str.split('.').str[:2].str.join('.')
df['sender_receiver_pair'] = df['sender_account'].astype(str) + '_' + df['receiver_account'].astype(str)
df['is_first_sender_receiver_tx'] = (df.groupby('sender_receiver_pair').cumcount() == 0).astype(int)

def get_time_of_day(hour):
    if 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 22:
        return 'evening'
    else:
        return 'night'
df['time_of_day'] = df['transaction_hour'].apply(get_time_of_day)

df_indexed = df.set_index('timestamp')

# Rolling window aggregates
sender_group = df_indexed.groupby('sender_account')
df['sender_1h_transaction_count'] = sender_group['amount'].rolling('1h').count().shift(1).values
df['sender_24h_transaction_count'] = sender_group['amount'].rolling('24h').count().shift(1).values
df['sender_7d_transaction_count'] = sender_group['amount'].rolling('7d').count().shift(1).values
df['sender_24h_avg_amount'] = sender_group['amount'].rolling('24h').mean().shift(1).values
df['account_24h_sum_amount'] = sender_group['amount'].rolling('24h').sum().shift(1).values
df['account_24h_max_amount'] = sender_group['amount'].rolling('24h').max().shift(1).values
df['account_24h_std_amount'] = sender_group['amount'].rolling('24h').std().shift(1).values

receiver_group = df_indexed.groupby('receiver_account')
df['receiver_24h_transaction_count'] = receiver_group['amount'].rolling('24h').count().shift(1).values
df['receiver_24h_unique_senders'] = receiver_group['sender_account_codes'].rolling('24h').apply(lambda x: x.nunique(), raw=False).shift(1).values
df['receiver_24h_sum_amount'] = receiver_group['amount'].rolling('24h').sum().shift(1).values
df['receiver_24h_max_amount'] = receiver_group['amount'].rolling('24h').max().shift(1).values

df['device_account_count_24h'] = df_indexed.groupby('device_hash')['sender_account_codes'].rolling('24h').apply(lambda x: x.nunique(), raw=False).shift(1).values
df['account_device_count_24h'] = df_indexed.groupby('sender_account')['device_hash_codes'].rolling('24h').apply(lambda x: x.nunique(), raw=False).shift(1).values
df['sender_24h_unique_receivers'] = sender_group['receiver_account_codes'].rolling('24h').apply(lambda x: x.nunique(), raw=False).shift(1).values

# Enhanced Aggregations: IP Address and Subnet
ip_group = df_indexed.groupby('ip_address')
subnet_group = df_indexed.groupby('ip_subnet')
df['ip_24h_transaction_count'] = ip_group['amount'].rolling('24h').count().shift(1).values
df['subnet_24h_transaction_count'] = subnet_group['amount'].rolling('24h').count().shift(1).values
df['subnet_24h_unique_accounts'] = subnet_group['sender_account_codes'].rolling('24h').apply(lambda x: x.nunique(), raw=False).shift(1).values

# Expanding window unique counts
def expanding_unique_count(df, group_col, target_col):
    codes, _ = pd.factorize(df[target_col])
    df[f'{target_col}_codes'] = codes
    expanding_max = df.groupby(group_col)[f'{target_col}_codes'].transform(lambda x: x.expanding().max())
    result = (expanding_max + 1).shift(1)
    df.drop(columns=[f'{target_col}_codes'], inplace=True)
    return result

df['receiver_unique_sender_devices'] = expanding_unique_count(df, 'receiver_account', 'device_hash')
df['device_unique_accounts'] = expanding_unique_count(df, 'device_hash', 'sender_account')
df['ip_unique_accounts'] = expanding_unique_count(df, 'ip_address', 'sender_account')

# Expanding means
for group, feature in [('sender_account', 'time_since_last_transaction'),
                       ('merchant_category', 'amount'),
                       ('sender_account', 'amount'),
                       ('sender_account', 'transaction_hour')]:
    numerator = df.groupby(group)[feature].cumsum().shift(1)
    denominator = df.groupby(group).cumcount()
    new_col_name = f"historical_avg_{feature}_by_{group}"
    if group == 'sender_account' and feature == 'time_since_last_transaction':
        new_col_name = 'avg_time_between_transactions'
    elif group == 'merchant_category' and feature == 'amount':
        new_col_name = 'merchant_historical_avg_amount'
    elif group == 'sender_account' and feature == 'amount':
        new_col_name = 'account_historical_avg_amount'
    elif group == 'sender_account' and feature == 'transaction_hour':
        new_col_name = 'account_historical_avg_hour'
    df[new_col_name] = numerator / (denominator + 1e-6)

# Robust historical baseline using median
df['account_historical_median_amount'] = df.groupby('sender_account')['amount'].transform(
    lambda x: x.shift(1).expanding().median()
)
    
df['receiver_historical_avg_amount'] = df.groupby('receiver_account')['amount'].transform(
    lambda x: x.shift(1).expanding().mean()
)
df['amount_vs_receiver_avg'] = df['amount'] / (df['receiver_historical_avg_amount'] + 1e-6)

# Historical fraud rate (leakage warning)
fraud_numerator = df.groupby('location')['is_fraud'].cumsum().shift(1)
fraud_denominator = df.groupby('location').cumcount()
df['location_historical_fraud_rate'] = fraud_numerator / (fraud_denominator + 1e-6)

# Benford's law proxy
df['amount_first_digit'] = df['amount'].astype(str).str[0].astype(int)
df['is_high_first_digit'] = df['amount_first_digit'].isin([7, 8, 9]).astype(int)
high_digit_numerator = df.groupby('sender_account')['is_high_first_digit'].cumsum().shift(1)
high_digit_denominator = df.groupby('sender_account').cumcount()
df['account_high_first_digit_ratio'] = high_digit_numerator / (high_digit_denominator + 1e-6)

# Receiver's last known device (leakage fix)
device_history = df[['sender_account', 'timestamp', 'device_hash']].dropna().copy()
device_history = device_history.rename(columns={'sender_account': 'receiver_account'})
df = pd.merge_asof(
    df,
    device_history,
    on='timestamp',
    by='receiver_account',
    direction='backward'
)
df['receiver_device_total_users'] = expanding_unique_count(df, 'device_hash_y', 'sender_account')
df.rename(columns={'device_hash_x': 'device_hash'}, inplace=True)
df.drop(columns=['device_hash_y'], inplace=True)

# Transaction roundness and amount nuances
df['is_amount_round_100'] = (df['amount'] % 100 == 0).astype(int)
df['is_amount_round_1000'] = (df['amount'] % 1000 == 0).astype(int)
df['amount_cents'] = (df['amount'] * 100 % 100).astype(int)
df['has_cents'] = (df['amount_cents'] != 0).astype(int)
df['is_micro_transaction'] = (df['amount'] < 1.00).astype(int)

# Behavioral timing variability
df['time_between_tx_std_24h'] = df_indexed.groupby('sender_account')['time_since_last_transaction'].rolling('24h').std().shift(1).values

# Context-specific timing
df['time_since_last_tx_by_merchant'] = df.groupby(['sender_account', 'merchant_category'])['timestamp'].diff().dt.total_seconds()

# Night-time activity
df['is_night_transaction'] = df['transaction_hour'].between(1, 5, inclusive='both').astype(int)

# Mule account proxies
df['fan_out_ratio_24h'] = df['sender_24h_unique_receivers'] / (df['sender_24h_transaction_count'] + 1e-6)
df['fan_in_ratio_24h'] = df['receiver_24h_unique_senders'] / (df['receiver_24h_transaction_count'] + 1e-6)

# New relationship detection
df['sender_receiver_pair'] = df['sender_account'].astype(str) + '_' + df['receiver_account'].astype(str)
df['is_first_sender_receiver_tx'] = (df.groupby('sender_receiver_pair').cumcount() == 0).astype(int)

# "Age" and Relationship History Features
df['account_first_tx_timestamp'] = df.groupby('sender_account')['timestamp'].transform('min')
df['time_since_account_creation'] = (df['timestamp'] - df['account_first_tx_timestamp']).dt.total_seconds() / 3600 # in hours
df['sender_receiver_tx_count'] = df.groupby('sender_receiver_pair').cumcount()
df['is_first_tx_for_receiver'] = (df.groupby('receiver_account').cumcount() == 0).astype(int)
df['receiver_first_tx_timestamp'] = df.groupby('receiver_account')['timestamp'].transform('min')
df['time_since_receiver_creation'] = (df['timestamp'] - df['receiver_first_tx_timestamp']).dt.total_seconds() / 3600
df['sender_receiver_first_tx'] = df.groupby(['sender_account', 'receiver_account'])['timestamp'].transform('min')
df['relationship_duration_hours'] = (df['timestamp'] - df['sender_receiver_first_tx']).dt.total_seconds() / 3600

# Advanced Aggregate & Interaction Features
merchant_group = df.groupby('merchant_category')['amount']
df['merchant_historical_std_amount'] = merchant_group.transform(lambda x: x.shift(1).expanding().std())
df['amount_merchant_z_score'] = (df['amount'] - df['merchant_historical_avg_amount']) / (df['merchant_historical_std_amount'] + 1e-6)
df['time_since_last_vs_avg_ratio'] = df['time_since_last_transaction'] / (df['avg_time_between_transactions'] + 1e-6)

# Account Age and Velocity Features
df['sender_new_receiver_count_24h'] = df_indexed.groupby('sender_account')['is_first_sender_receiver_tx'].rolling('24h').sum().shift(1).values

# Derived and interaction features
df['velocity_change_1h_vs_24h'] = df['sender_1h_transaction_count'] / (df['sender_24h_transaction_count'] + 1e-6)
df['amount_to_merchant_avg_ratio'] = df['amount'] / (df['merchant_historical_avg_amount'] + 1e-6)
df['amount_deviation_from_merchant_avg'] = df['amount'] - df['merchant_historical_avg_amount']
df['amount_vs_account_avg'] = df['amount'] / (df['account_historical_avg_amount'] + 1e-6)
df['amount_vs_account_median_ratio'] = df['amount'] / (df['account_historical_median_amount'] + 1e-6)
df['hour_deviation_from_avg'] = (df['transaction_hour'] - df['account_historical_avg_hour']).abs()
df['type_device_interaction'] = df['transaction_type'] + '_' + df['device_used']
df['channel_merchant_interaction'] = df['payment_channel'] + '_' + df['merchant_category']
df['is_first_transaction_for_account'] = (df.groupby('sender_account').cumcount() == 0).astype(int)
df['account_device_pair'] = df['sender_account'] + '_' + df['device_hash']
df['is_first_account_device_pair'] = (df.groupby('account_device_pair').cumcount() == 0).astype(int)
df['prev_location'] = df.groupby('sender_account')['location'].shift(1)
df['is_new_location'] = (df['location'] != df['prev_location']).astype(int)
df['amount_x_spending_deviation'] = df['amount'] * df['spending_deviation_score']
df['amount_x_velocity'] = df['amount'] * df['velocity_score']
df['geo_x_spending_deviation'] = df['geo_anomaly_score'] * df['spending_deviation_score']

# Advanced Ratio, Difference, and Interaction Features
df['sender_1h_sum_amount'] = sender_group['amount'].rolling('1h').sum().shift(1).values
df['amount_ratio_1h_vs_24h'] = df['sender_1h_sum_amount'] / (df['account_24h_sum_amount'] + 1e-6)
df['amount_deviation_from_account_avg'] = df['amount'] - df['account_historical_avg_amount']

# Historical average spend for a specific user-merchant pair
df['account_merchant_avg_spend'] = df.groupby(['sender_account', 'merchant_category'])['amount'].transform(
    lambda x: x.shift(1).expanding().mean()
)
df['amount_vs_account_merchant_avg'] = df['amount'] / (df['account_merchant_avg_spend'] + 1e-6)

# Graph/Network-Based Features
device_map = df.sort_values('timestamp').groupby('sender_account')['device_hash'].last()
df['receiver_last_known_device'] = df['receiver_account'].map(device_map)
df['sender_receiver_device_same'] = (df['device_hash'] == df['receiver_last_known_device']).astype(int)

# Shared device detection (Network/Graph feature)
device_to_users = {}
historical_device_users = []
for device, user in zip(df['device_hash'], df['sender_account']):
    if device not in device_to_users:
        device_to_users[device] = set()
    device_to_users[device].add(user)
    historical_device_users.append(device_to_users[device].copy())
df['historical_users_for_device'] = historical_device_users
df['historical_users_for_device'] = df['historical_users_for_device'].shift(1)
# Using a loop for clarity on the potentially slow operation
is_shared_list = []
for receiver, users in zip(df['receiver_account'], df['historical_users_for_device']):
    if users and isinstance(users, set):
        is_shared_list.append(receiver in users)
    else:
        is_shared_list.append(False)
df['is_shared_device_receiver'] = pd.Series(is_shared_list).astype(int)

# Frequency Encoding for High-Cardinality Categorical Features
for col in ['type_device_interaction', 'channel_merchant_interaction', 'location', 'merchant_category', 'ip_subnet']:
    freq_map = df[col].value_counts(normalize=True)
    df[f'{col}_freq_encoded'] = df[col].map(freq_map)

# Handle NaN values created by shifts and rolling windows
# A simple fillna(0) is often effective as it signals "no prior history"
new_feature_cols = [
    'sender_1h_transaction_count', 'sender_24h_transaction_count', 'sender_7d_transaction_count',
    'sender_24h_avg_amount', 'account_24h_sum_amount', 'account_24h_max_amount', 'account_24h_std_amount',
    'receiver_24h_transaction_count', 'receiver_24h_unique_senders', 'receiver_24h_sum_amount',
    'receiver_24h_max_amount', 'device_account_count_24h', 'account_device_count_24h',
    'sender_24h_unique_receivers', 'receiver_unique_sender_devices', 'device_unique_accounts',
    'ip_unique_accounts', 'avg_time_between_transactions', 'merchant_historical_avg_amount',
    'account_historical_avg_amount', 'account_historical_avg_hour', 'location_historical_fraud_rate',
    'account_high_first_digit_ratio', 'receiver_device_total_users', 'time_between_tx_std_24h',
    'fan_out_ratio_24h', 'fan_in_ratio_24h', 'velocity_change_1h_vs_24h',
    'amount_to_merchant_avg_ratio', 'amount_deviation_from_merchant_avg', 'amount_vs_account_avg',
    'hour_deviation_from_avg', 'ip_24h_transaction_count', 'subnet_24h_transaction_count',
    'subnet_24h_unique_accounts', 'time_since_account_creation', 'sender_1h_sum_amount',
    'amount_ratio_1h_vs_24h', 'amount_deviation_from_account_avg', 'account_merchant_avg_spend',
    'amount_vs_account_merchant_avg', 'receiver_historical_avg_amount', 'amount_vs_receiver_avg',
    'time_since_receiver_creation', 'relationship_duration_hours',
    'merchant_historical_std_amount', 'amount_merchant_z_score', 'time_since_last_vs_avg_ratio',
    'sender_new_receiver_count_24h', 'account_historical_median_amount', 'amount_vs_account_median_ratio',
    'time_since_last_tx_by_merchant'
]
df[new_feature_cols] = df[new_feature_cols].fillna(0)

# Cleanup
df.drop(columns=['sender_receiver_pair', 'account_device_pair', 'prev_location', 
                 'account_first_tx_timestamp', 'ip_subnet', 'receiver_first_tx_timestamp',
                 'sender_receiver_first_tx', 'receiver_last_known_device',
                 'historical_users_for_device', 'receiver_account_codes'], inplace=True)

In [11]:
# TODO: Optimize the feature engineering process further, possibly by parallelizing rolling operations or using more efficient libraries like Polars.

In [11]:
df.to_csv('../datasets/financial_fraud_feature_engineer.csv', index=False)